In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import plotly
import plotly.graph_objs as go

city_data =  ("data/city_data.csv")
ride_data =  ("data/ride_data.csv")

city_df = pd.read_csv(city_data)
ride_df = pd.read_csv(ride_data)

data = pd.merge(city_df, ride_df, how='inner', on='city')
# data=merge_df.drop_duplicates('city')



In [ ]:
data

In [ ]:
city_total = data.groupby(['type','city']).agg({'driver_count':'mean',
                                                   'fare':'mean',
                                                   'ride_id':'count'})

city_report = city_total.rename(columns = {"driver_count": "Total Drivers",
                                               "fare": "average fare",
                                               "ride_id": "total rides"})



## Summary of Data 

In [2]:
## groups by type and city to give me total rides total avg fare and total drivers then changes coloum 

rides_total = data.groupby(['type', 'city'], as_index=False)['ride_id'].count()
avg_fare = data.groupby(['type', 'city'], as_index=False)['fare'].mean()
driver_total = data.groupby(['type', 'city'], as_index=False)['driver_count'].mean()

# Rename colums 
Rides_total= rides_total.rename({'ride_id':'total rides'},axis=1)
Avg_fare = avg_fare.rename({'fare':'average fare'},axis=1)
Driver_total= driver_total.rename({'driver_count':'Total Drivers'},axis=1)

In [3]:
##  Merge two data frames from above in to one 

combined_df = pd.merge(Avg_fare,Rides_total, how='inner', on='city',
                      suffixes=('_m','_n'), indicator=True)

combined_df['type']= combined_df['type_m'].where(combined_df['type_n'].isnull(),
                                                combined_df['type_n'])

combined_df.drop(['type_m','type_n','_merge'], axis=1, inplace=True)
# combined =combined_df.drop_duplicates('type', keep = 'first')



In [4]:
## create final data frame adding Driver total 

df = pd.merge(combined_df,Driver_total, how ='inner', on = 'city',
                      suffixes=('_m','_n'), indicator=True)
df['type']= df['type_m'].where(df['type_n'].isnull(),
                                                df['type_n'])
# df[['type','city','city_m','city_n']]
df.drop(['type_m','type_n','_merge'], axis=1, inplace=True)



df

city  average fare  total rides  Total Drivers   type
0           Bradshawfurt     40.064000           10              7  Rural
1              Garzaport     24.123333            3              7  Rural
2         Harringtonfort     33.470000            6              4  Rural
3            Jessicaport     36.013333            6              1  Rural
4             Lake Jamie     34.358333            6              4  Rural
..                   ...           ...          ...            ...    ...
115  West Patrickchester     28.233125           16             25  Urban
116          West Robert     25.123871           31             39  Urban
117     West Samuelburgh     21.767600           25             73  Urban
118         Williamsstad     24.362174           23             59  Urban
119         Williamsview     26.599000           20             46  Urban

[120 rows x 5 columns]

<h3>Separate by type <h3>

In [14]:

Suburban = df.loc[df["type"]=="Suburban",:]
Urban = df.loc[df["type"]=="Urban",:]
Rural = df.loc[df["type"]=="Rural",:]



fig = go.Figure()

fig.add_trace(go.Scatter(x=Urban["total rides"], y=Urban["average fare"], name="Urban", hovertext=Urban["city"],
                         
                         marker=dict(size=Urban["Total Drivers"], color='rosybrown'),
                 
                        mode="markers"))

fig.add_trace(go.Scatter(x=Suburban["total rides"], y=Suburban["average fare"], name="Suburban", hovertext=Suburban["city"],marker=dict(
                size=Suburban["Total Drivers"],
                color='lightgreen'), 
                mode="markers"))

fig.add_trace(go.Scatter(x=Rural["total rides"], y=Rural["average fare"],name="Rural", hovertext= Rural["city"],marker=dict(
                size=Rural["Total Drivers"],
                color='red'), 
                mode="markers"))

fig.update_layout(autosize=False,width=1300,height=850,title = 'Ride Sharing; Bubble size is by Total Drivers',
          xaxis=dict(title='Total Rides'), yaxis=dict(title='Average Fare'))

# plotly.offline.plot(fig)

'temp-plot.html'

In [21]:
type_fare= data.groupby(['type'], as_index=False)['fare'].sum()

colors = ['gold', 'lightblue', 'orange']

fig = go.Figure(data=[go.Pie(labels=['Rural', 'Suburban','Urban'],
                            values=type_fare['fare'])])

fig.update_traces(hoverinfo='label+value',textinfo='percent',
                 textfont_size=20, marker=dict(colors=colors,
                                              line=dict(color='#000000',
                                                       width=2)))
fig.update_layout(title_text="Total Fare by City Type", autosize=False, height=700, width=700)

plotly.offline.plot(fig, filename='pie.html')

'pie.html'

In [16]:
rides_sum= data.groupby(['type'], as_index=False)['ride_id'].count()

colors = ['gold', 'aliceblue', 'red']

fig = go.Figure(data=[go.Pie(labels=['Rural', 'Suburban','Urban'],
                            values=rides_sum['ride_id'])])

fig.update_traces(hoverinfo='label+value',textinfo='percent',
                 textfont_size=20, marker=dict(colors=colors,
                                              line=dict(color='#000000',
                                                       width=2)))
fig.update_layout(title_text="Total Rides by City Type")

fig.show()

# Create Bar Graphs 